In [1]:
import scipy.stats as ss
import numpy as np
import cvxpy as cp
import simpy as smp

In [37]:
#Offline with 5 cooks

missed_clients = 0
price = 278
cost = 140
profits = []

coefficients_months = [0.35 + 0.05*abs(k-6) for k in range(12)]

for k in range(12):

    arrival_ticks = np.zeros(300000)
    served_clients = []
    demand = max(0, coefficients_months[k]*(60000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)

    groups = ss.rv_discrete(values=([1,2,3,4,5], [0.3,0.4,0.2,0.05,0.05])).rvs(size = round(demand/2.15))
    t =  np.cumsum(groups)
    x = ss.expon(scale=3.2).rvs(size = round(demand/2.15))
    arrival_ticks[t-groups]=x
    arrival_ticks=np.trim_zeros(arrival_ticks, trim='b')
    arrivals = np.cumsum(arrival_ticks)
    service_tick = 3
    #print(demand)
    #print(coefficients_months[k])
    def source(env, waiting_room, servers):
        i = 0
        while i<=len(arrival_ticks)-1:
            arrival_tick = arrival_ticks[i]
            c = customer(env, i, waiting_room, servers, served_clients)
            yield env.timeout(arrival_tick)
            env.process(c)
            i += 1

    def customer(env, i, waiting_room, serv, served_clients):
          arrive = env.now
          
          #print(f'{i} arrived at {arrive}')
          arrival_times[i] = arrive
          with waiting_room.request() as wait_req:
                  yield wait_req
                  with serv.request() as serv_req:
                          yield serv_req
                          service_tick = 3
                          srv_comp = env.now + service_tick
                          wait_times.append(srv_comp-arrive)
                          served_clients.append(1)
                          yield env.timeout(service_tick)
                          
                          #print(f'{i} left at {env.now}')


    n_servers = 5
    arrival_times, departure_times, wait_times = {}, {}, []
    env = smp.Environment()
    waiting_room = smp.Resource(env, capacity=10)
    server = smp.Resource(env, capacity=n_servers)
    env.process(source(env, waiting_room, server))
    TT = 41000
    env.run(until = TT)
    profits.append(sum(served_clients) * (price-cost) - n_servers*170000 - 600000)
    missed_clients += np.sum(np.array(wait_times) > 10) + demand - sum(served_clients)
np.sum(profits)

12664956

default setting: starting demand = season_coefficient x 60000, waiting_room_capacity = 20, waiting_time_until_unhappy = 10, 
price = 278, cost = 140, service_tick = 3, FC = 600000, стоимость повара на смене = 170000

In [33]:
#Offline with advertising (with 5 cooks)
missed_clients = 0
price = 278
cost = 140
profits = []
cost_of_advertising = 250000

coefficients_months = [0.35 + 0.05*abs(k-6) for k in range(12)]

for k in range(12):

    arrival_ticks = np.zeros(300000)
    served_clients = []
    demand = max(0, coefficients_months[k]*(80000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)

    groups = ss.rv_discrete(values=([1,2,3,4,5], [0.3,0.4,0.2,0.05,0.05])).rvs(size = round(demand/2.15))
    t =  np.cumsum(groups)
    x = ss.expon(scale=3.2).rvs(size = round(demand/2.15))
    arrival_ticks[t-groups]=x
    arrival_ticks=np.trim_zeros(arrival_ticks, trim='b')
    arrivals = np.cumsum(arrival_ticks)
    service_tick = 3
    #print(demand)
    #print(coefficients_months[k])
    def source(env, waiting_room, servers):
        i = 0
        while i<=len(arrival_ticks)-1:
            arrival_tick = arrival_ticks[i]
            c = customer(env, i, waiting_room, servers, served_clients)
            yield env.timeout(arrival_tick)
            env.process(c)
            i += 1

    def customer(env, i, waiting_room, serv, served_clients):
          arrive = env.now
          
          #print(f'{i} arrived at {arrive}')
          arrival_times[i] = arrive
          with waiting_room.request() as wait_req:
                  yield wait_req
                  with serv.request() as serv_req:
                          yield serv_req
                          service_tick = 3
                          srv_comp = env.now + service_tick
                          wait_times.append(srv_comp-arrive)
                          served_clients.append(1)
                          yield env.timeout(service_tick)
                          
                          #print(f'{i} left at {env.now}')


    n_servers = 5
    arrival_times, departure_times, wait_times = {}, {}, []
    env = smp.Environment()
    waiting_room = smp.Resource(env, capacity=20)
    server = smp.Resource(env, capacity=n_servers)
    env.process(source(env, waiting_room, server))
    TT = 41000
    env.run(until = TT)
    profits.append(sum(served_clients) * (price-cost) - n_servers * 170000-600000-cost_of_advertising)
    missed_clients += np.sum(np.array(wait_times) > 10) + demand - sum(served_clients)
np.sum(profits)

4811772

In [34]:
#Online with delivery club (with 5 cooks)
missed_clients = 0
price = 278
cost = 140
profits = []


coefficients_inverse = [1-coefficients_months[k] for k in range(12)]
for k in range(12):



    demand = max(0, coefficients_inverse[k]*(60000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)
    served_clients = []
    arrival_ticks = ss.expon(scale=3.2).rvs(size = round(demand))
    service_tick = 3
    #print(demand)
    #print(missed_clients)
    #print(coefficients_inverse[k])
    def source(env, waiting_room, servers):
        i = 0
        while i<=len(arrival_ticks)-1:
            arrival_tick = arrival_ticks[i]
            c = customer(env, i, waiting_room, servers)
            yield env.timeout(arrival_tick)
            env.process(c)
            i += 1

    def customer(env, i, waiting_room, serv):
          arrive = env.now
          #print(f'{i} arrived at {arrive}')
          arrival_times[i] = arrive
          with waiting_room.request() as wait_req:
                  yield wait_req
                  with serv.request() as serv_req:
                          yield serv_req
                          service_tick = 3
                          srv_comp = env.now + service_tick
                          wait_times.append(srv_comp-arrive)
                          
                          yield env.timeout(service_tick)
                          #print(f'{i} left at {env.now}')


    n_servers = 5
    arrival_times, departure_times, wait_times = {}, {}, []
    env = smp.Environment()
    waiting_room = smp.Resource(env, capacity=float("inf"))
    server = smp.Resource(env, capacity=n_servers)
    env.process(source(env, waiting_room, server))
    TT = 41000
    env.run(until = TT)
    profits.append(round(len(arrival_ticks) * (price-cost) * 0.65 - n_servers*170000-300000))
    missed_clients += np.sum(np.array(wait_times) > 30)
np.sum(profits)

18504378

In [36]:
#Online with our couriers (with 5 cooks)
profits = []
price = 278
cost = 140
n_cooks = 5
n_couriers = 5
missed_clients = 0
p3_flow_times = [5, 10]
served_clients = []

for k in range(12):
    env = smp.Environment()
    demand = max(0, coefficients_inverse[k]*(60000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)
    staff = [smp.Resource(env, capacity=n_cooks), smp.Resource(env, capacity=n_couriers)]
    orders = [0,1]

    arrival_ticks = ss.expon(scale=3.2).rvs(size = round(demand))

    def customer(env, i, waiting_room, staff, served_clients):
        arrive = env.now
        #print(f'{i} arrived at {arrive}')
        arrival_times[i] = arrive
        with waiting_room.request() as wait_req:
            yield wait_req
            for j in range(2):
                if j == 0:
                    doc = orders[j]
                    with staff[doc].request() as req:
                        #print(f'Patient {i} requests burgers from {doc} at {env.now}')
                        yield req
                        #print(f'Patient {i} is getting burgers cooked by {doc} at {env.now}')
                        yield env.timeout(p3_flow_times[doc])
                        #print(f'Patient {i} gets burgers cooked by {doc} at {env.now}')
                else:
                    doc = orders[j]
                    with staff[doc].request() as req:
                        #print(f'Patient {i} requests burgers delivery by {doc} at {env.now}')
                        yield req
                        #print(f'Patient {i} is getting burgers delivered by {doc} at {env.now}')
                        yield env.timeout(p3_flow_times[doc])
                        #print(f'Patient {i} finally gets burgers from {doc} at {env.now}')
                        wait_times.append(env.now-arrive)
                        
    def source(env, waiting_room, staff):
        i = 0
        while i<=len(arrival_ticks)-1:
            arrival_tick = arrival_ticks[i]
            c = customer(env, i, waiting_room, staff, served_clients)
            yield env.timeout(arrival_tick)
            env.process(c)
            i += 1

    arrival_times, departure_times, wait_times = {}, {}, []
    waiting_room = smp.Resource(env, capacity=float("inf"))

    env.process(source(env, waiting_room, staff))
    TT = 41000
    env.run(until = TT)
    missed_clients += np.sum(np.array(wait_times) > 30)
    profits.append(len(arrival_ticks) * (price-cost) * 0.8 - n_cooks*170000 - n_couriers*170000 - 300000)
np.sum(profits)

15704808.000000004

default setting: starting demand = season_coefficient x 60000, waiting_room_capacity = 20, waiting_time_until_unhappy = 10

In [3]:
#Default settings offline optimization
for n in range(1, 15):
    missed_clients = 0
    price = 278
    cost = 140
    profits = []
    n_servers = n
    
    coefficients_months = [0.35 + 0.05*abs(k-6) for k in range(12)]

    for k in range(12):

        arrival_ticks = np.zeros(300000)
        served_clients = []
        demand = max(0, coefficients_months[k]*(60000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)

        groups = ss.rv_discrete(values=([1,2,3,4,5], [0.3,0.4,0.2,0.05,0.05])).rvs(size = round(demand/2.15))
        t =  np.cumsum(groups)
        x = ss.expon(scale=3.2).rvs(size = round(demand/2.15))
        arrival_ticks[t-groups]=x
        arrival_ticks=np.trim_zeros(arrival_ticks, trim='b')
        arrivals = np.cumsum(arrival_ticks)
        service_tick = 3
        #print(demand)
        #print(coefficients_months[k])
        def source(env, waiting_room, servers):
            i = 0
            while i<=len(arrival_ticks)-1:
                arrival_tick = arrival_ticks[i]
                c = customer(env, i, waiting_room, servers, served_clients)
                yield env.timeout(arrival_tick)
                env.process(c)
                i += 1

        def customer(env, i, waiting_room, serv, served_clients):
              arrive = env.now

              #print(f'{i} arrived at {arrive}')
              arrival_times[i] = arrive
              with waiting_room.request() as wait_req:
                      yield wait_req
                      with serv.request() as serv_req:
                              yield serv_req
                              service_tick = 3
                              srv_comp = env.now + service_tick
                              wait_times.append(srv_comp-arrive)
                              served_clients.append(1)
                              yield env.timeout(service_tick)

                              #print(f'{i} left at {env.now}')


        
        arrival_times, departure_times, wait_times = {}, {}, []
        env = smp.Environment()
        waiting_room = smp.Resource(env, capacity=20)
        server = smp.Resource(env, capacity=n_servers)
        env.process(source(env, waiting_room, server))
        TT = 41000
        env.run(until = TT)
        profits.append(sum(served_clients) * (price-cost) - n_servers*170000 - 600000)
        missed_clients += np.sum(np.array(wait_times) > 10) + demand - sum(served_clients)
    print(f'When we have {n} cooks we get ' + str(np.sum(profits)))

When we have 1 cooks we get -7354782
When we have 2 cooks we get -7518120
When we have 3 cooks we get 3470598
When we have 4 cooks we get 13078488
When we have 5 cooks we get 12530820
When we have 6 cooks we get 11118168
When we have 7 cooks we get 8127762
When we have 8 cooks we get 6464502
When we have 9 cooks we get 4470180
When we have 10 cooks we get 3780510
When we have 11 cooks we get 1417728
When we have 12 cooks we get 308814
When we have 13 cooks we get -4216152
When we have 14 cooks we get -5090328


Default settings online optimization with delivery club couriers:
waiting_room_capacity = infinity, delivery club commision = 35%, waiting_time_until_unhappy = 30, delivery_time = 10, FC = 300

In [30]:
#Default settings online optimization with delivery club couriers
for n in range(1, 8):
    missed_clients = 0
    price = 278
    cost = 140
    profits = []
    n_servers = n

    coefficients_inverse = [1-coefficients_months[k] for k in range(12)]
    for k in range(12):



        demand = max(0, coefficients_inverse[k]*(60000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)
        served_clients = []
        arrival_ticks = ss.expon(scale=3.2).rvs(size = round(demand))
        service_tick = 3
        #print(demand)
        #print(missed_clients)
        #print(coefficients_inverse[k])
        def source(env, waiting_room, servers):
            i = 0
            while i<=len(arrival_ticks)-1:
                arrival_tick = arrival_ticks[i]
                c = customer(env, i, waiting_room, servers)
                yield env.timeout(arrival_tick)
                env.process(c)
                i += 1

        def customer(env, i, waiting_room, serv):
              arrive = env.now
              #print(f'{i} arrived at {arrive}')
              arrival_times[i] = arrive
              with waiting_room.request() as wait_req:
                      yield wait_req
                      with serv.request() as serv_req:
                              yield serv_req
                              service_tick = 3
                              srv_comp = env.now + service_tick
                              wait_times.append(srv_comp-arrive+10)

                              yield env.timeout(service_tick)
                              #print(f'{i} left at {env.now}')


        
        arrival_times, departure_times, wait_times = {}, {}, []
        env = smp.Environment()
        waiting_room = smp.Resource(env, capacity=100)
        server = smp.Resource(env, capacity=n_servers)
        env.process(source(env, waiting_room, server))
        TT = 41000
        env.run(until = TT)
        profits.append(round(len(arrival_ticks) * (price-cost) * 0.65 - n_servers*170000-300000))
        missed_clients += np.sum(np.array(wait_times) > 30)
    print(f'When we have {n} cooks we get ' + str(np.sum(profits)))

When we have 1 cooks we get 2341955
When we have 2 cooks we get 24620971
When we have 3 cooks we get 22585903
When we have 4 cooks we get 20519801
When we have 5 cooks we get 18516041
When we have 6 cooks we get 16459446
When we have 7 cooks we get 14406439


Default settings online optimization with our couriers:
waiting_room_capacity = infinity, delivery club commision = 20%, waiting_time_until_unhappy = 30, delivery_time = 10, FC = 300

In [5]:
#Default settings online optimization with our couriers
for n1 in range(1,8):
    profits_for_n_cooks = []
    for n2 in range(1,8):
        profits = []
        price = 278
        cost = 140
        n_cooks = n1
        n_couriers = n2
        missed_clients = 0
        p3_flow_times = [5, 10]
        served_clients = []

        for k in range(12):
            env = smp.Environment()
            demand = max(0, coefficients_inverse[k]*(60000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)
            staff = [smp.Resource(env, capacity=n_cooks), smp.Resource(env, capacity=n_couriers)]
            orders = [0,1]
            arrival_ticks = ss.expon(scale=3.2).rvs(size = round(demand))

            def customer(env, i, waiting_room, staff, served_clients):
                arrive = env.now
                #print(f'{i} arrived at {arrive}')
                arrival_times[i] = arrive
                with waiting_room.request() as wait_req:
                    yield wait_req
                    for j in range(2):
                        if j == 0:
                            doc = orders[j]
                            with staff[doc].request() as req:
                                #print(f'Patient {i} requests burgers from {doc} at {env.now}')
                                yield req
                                #print(f'Patient {i} is getting burgers cooked by {doc} at {env.now}')
                                yield env.timeout(p3_flow_times[doc])
                                #print(f'Patient {i} gets burgers cooked by {doc} at {env.now}')
                        else:
                            doc = orders[j]
                            with staff[doc].request() as req:
                                #print(f'Patient {i} requests burgers delivery by {doc} at {env.now}')
                                yield req
                                #print(f'Patient {i} is getting burgers delivered by {doc} at {env.now}')
                                yield env.timeout(p3_flow_times[doc])
                                #print(f'Patient {i} finally gets burgers from {doc} at {env.now}')
                                wait_times.append(env.now-arrive)

            def source(env, waiting_room, staff):
                i = 0
                while i<=len(arrival_ticks)-1:
                    arrival_tick = arrival_ticks[i]
                    c = customer(env, i, waiting_room, staff, served_clients)
                    yield env.timeout(arrival_tick)
                    env.process(c)
                    i += 1

            arrival_times, departure_times, wait_times = {}, {}, []
            waiting_room = smp.Resource(env, capacity=100)

            env.process(source(env, waiting_room, staff))
            TT = 41000
            env.run(until = TT)
            missed_clients += np.sum(np.array(wait_times) > 30)
            profits.append(len(arrival_ticks) * (price-cost) * 0.8 - n_cooks*170000 - n_couriers*170000 - 300000)
        profits_for_n_cooks.append((n2, np.sum(profits)))
    profit_x = -1000000
    for i in range(7):
        if profits_for_n_cooks[i][1] > profit_x:
            profit_x = profits_for_n_cooks[i][1]
    for i in range(7):
        if profits_for_n_cooks[i][1] == profit_x:
            print(f'We get the best profit for {n1} cooks when we have {i+1} couriers and get ' + str(profit_x))
            break

We get the best profit for 1 cooks when we have 1 couriers and get 6805694.400000002
We get the best profit for 2 cooks when we have 4 couriers and get 20449694.400000002
We get the best profit for 3 cooks when we have 5 couriers and get 19792646.4
We get the best profit for 4 cooks when we have 5 couriers and get 17762251.200000003
We get the best profit for 5 cooks when we have 5 couriers and get 15659875.200000001
We get the best profit for 6 cooks when we have 5 couriers and get 13677945.600000001
We get the best profit for 7 cooks when we have 5 couriers and get 11579875.2


Offline with advertisement: new base demand = 100000, cost_of_advertising = 250000

In [11]:
#Back to offline but we decided to advertise:
for n in range(1, 8):
    missed_clients = 0
    price = 278
    cost = 140
    profits = []
    cost_of_advertising = 250000
    
    coefficients_months = [0.35 + 0.05*abs(k-6) for k in range(12)]
    
    n_servers = n
    
    for k in range(12):

        arrival_ticks = np.zeros(300000)
        served_clients = []
        demand = max(0, coefficients_months[k]*(100000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)

        groups = ss.rv_discrete(values=([1,2,3,4,5], [0.3,0.4,0.2,0.05,0.05])).rvs(size = round(demand/2.15))
        t =  np.cumsum(groups)
        x = ss.expon(scale=2.5).rvs(size = round(demand/2.15))
        arrival_ticks[t-groups]=x
        arrival_ticks=np.trim_zeros(arrival_ticks, trim='b')
        arrivals = np.cumsum(arrival_ticks)
        service_tick = 3
        #print(demand)
        #print(coefficients_months[k])
        def source(env, waiting_room, servers):
            i = 0
            while i<=len(arrival_ticks)-1:
                arrival_tick = arrival_ticks[i]
                c = customer(env, i, waiting_room, servers, served_clients)
                yield env.timeout(arrival_tick)
                env.process(c)
                i += 1

        def customer(env, i, waiting_room, serv, served_clients):
              arrive = env.now

              #print(f'{i} arrived at {arrive}')
              arrival_times[i] = arrive
              with waiting_room.request() as wait_req:
                      yield wait_req
                      with serv.request() as serv_req:
                              yield serv_req
                              service_tick = 3
                              srv_comp = env.now + service_tick
                              wait_times.append(srv_comp-arrive)
                              served_clients.append(1)
                              yield env.timeout(service_tick)

                              #print(f'{i} left at {env.now}')


        
        arrival_times, departure_times, wait_times = {}, {}, []
        env = smp.Environment()
        waiting_room = smp.Resource(env, capacity=20)
        server = smp.Resource(env, capacity=n_servers)
        env.process(source(env, waiting_room, server))
        TT = 41000
        env.run(until = TT)
        profits.append(sum(served_clients) * (price-cost) - n_servers * 170000-600000-cost_of_advertising)
        missed_clients += np.sum(np.array(wait_times) > 10) + demand - sum(served_clients)
    print(f'When we have {n} cooks with advertising we get ' + str(np.sum(profits)))

When we have 1 cooks with advertising we get -10354092
When we have 2 cooks with advertising we get -10508184
When we have 3 cooks with advertising we get -7792566
When we have 4 cooks with advertising we get 7750566
When we have 5 cooks with advertising we get 12327942
When we have 6 cooks with advertising we get 10791642
When we have 7 cooks with advertising we get 8480886


In [31]:
#Offline optimization with capacity and/or waiting time increased
for waiting_time_critical in [10, 20, 30]:
    for capacity_of_room in [10, 20, 30]:
        for n in range(1, 8):
            
            missed_clients = 0
            price = 278
            cost = 140
            profits = []
            n_servers = n

            coefficients_months = [0.35 + 0.05*abs(k-6) for k in range(12)]

            for k in range(12):

                arrival_ticks = np.zeros(300000)
                served_clients = []
                demand = max(0, coefficients_months[k]*(60000 + ss.norm(scale = 100, loc = 0).rvs()) - missed_clients)

                groups = ss.rv_discrete(values=([1,2,3,4,5], [0.3,0.4,0.2,0.05,0.05])).rvs(size = round(demand/2.15))
                t =  np.cumsum(groups)
                x = ss.expon(scale=3.2).rvs(size = round(demand/2.15))
                arrival_ticks[t-groups]=x
                arrival_ticks=np.trim_zeros(arrival_ticks, trim='b')
                arrivals = np.cumsum(arrival_ticks)
                service_tick = 3
                #print(demand)
                #print(coefficients_months[k])
                def source(env, waiting_room, servers):
                    i = 0
                    while i<=len(arrival_ticks)-1:
                        arrival_tick = arrival_ticks[i]
                        c = customer(env, i, waiting_room, servers, served_clients)
                        yield env.timeout(arrival_tick)
                        env.process(c)
                        i += 1

                def customer(env, i, waiting_room, serv, served_clients):
                      arrive = env.now

                      #print(f'{i} arrived at {arrive}')
                      arrival_times[i] = arrive
                      with waiting_room.request() as wait_req:
                              yield wait_req
                              with serv.request() as serv_req:
                                      yield serv_req
                                      service_tick = 3
                                      srv_comp = env.now + service_tick
                                      wait_times.append(srv_comp-arrive)
                                      served_clients.append(1)
                                      yield env.timeout(service_tick)

                                      #print(f'{i} left at {env.now}')



                arrival_times, departure_times, wait_times = {}, {}, []
                env = smp.Environment()
                waiting_room = smp.Resource(env, capacity=capacity_of_room)
                server = smp.Resource(env, capacity=n_servers)
                env.process(source(env, waiting_room, server))
                TT = 41000
                env.run(until = TT)
                profits.append(sum(served_clients) * (price-cost) - n_servers * 170000 - 600000)
                missed_clients += np.sum(np.array(wait_times) > waiting_time_critical) + demand - sum(served_clients)
            print(f'When waiting time is {waiting_time_critical} and capacity is {capacity_of_room} and we have {n} cooks we get ' + str(np.sum(profits)))

When waiting time is 10 and capacity is 10 and we have 1 cooks we get -7354368
When waiting time is 10 and capacity is 10 and we have 2 cooks we get -7548066
When waiting time is 10 and capacity is 10 and we have 3 cooks we get 2935296
When waiting time is 10 and capacity is 10 and we have 4 cooks we get 12584586
When waiting time is 10 and capacity is 10 and we have 5 cooks we get 12658332
When waiting time is 10 and capacity is 10 and we have 6 cooks we get 11401896
When waiting time is 10 and capacity is 10 and we have 7 cooks we get 9538122
When waiting time is 10 and capacity is 20 and we have 1 cooks we get -7354368
When waiting time is 10 and capacity is 20 and we have 2 cooks we get -7507908
When waiting time is 10 and capacity is 20 and we have 3 cooks we get 2965794
When waiting time is 10 and capacity is 20 and we have 4 cooks we get 12563472
When waiting time is 10 and capacity is 20 and we have 5 cooks we get 13279470
When waiting time is 10 and capacity is 20 and we have 